In [1]:
import os

In [2]:
%pwd

'/Users/koushik/Desktop/MLProject-mlflow/research'

In [3]:

os.chdir("../")

In [4]:
%pwd

'/Users/koushik/Desktop/MLProject-mlflow'

In [ ]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
